# Перехресна валідація (Cross Validation)

## k-кратна перехресна валідація (k-fold Cross Validation)

Ця техніка розбиває дані на k підмножин (folds) і використовує k-1 підмножин для тренування моделі, а одну підмножину для тестування. Процес повторюється k разів, кожного разу використовуючи іншу підмножину для тестування.

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

X = np.array([[101, 202], [303, 404], [102, 204], [306, 408]])
y = np.array([1, 2, 3, 4])

In [14]:
X

array([[101, 202],
       [303, 404],
       [102, 204],
       [306, 408]])

In [15]:
?KFold

Init signature: KFold(n_splits=5, *, shuffle=False, random_state=None)
Docstring:     
K-Fold cross-validator.

Provides train/test indices to split data in train/test sets. Split
dataset into k consecutive folds (without shuffling by default).

Each fold is then used once as a validation while the k - 1 remaining
folds form the training set.

Read more in the :ref:`User Guide <k_fold>`.

For visualisation of cross-validation behaviour and
comparison between common scikit-learn split methods
refer to :ref:`sphx_glr_auto_examples_model_selection_plot_cv_indices.py`

Parameters
----------
n_splits : int, default=5
    Number of folds. Must be at least 2.

    .. versionchanged:: 0.22
        ``n_splits`` default value changed from 3 to 5.

shuffle : bool, default=False
    Whether to shuffle the data before splitting into batches.
    Note that the samples within each split will not be shuffled.

random_state : int, RandomState instance or None, default=None
    When `shuffle` is True, `

In [16]:
kf = KFold(n_splits=3)

In [17]:
kf.split(X)

<generator object _BaseKFold.split at 0x00000267A9911480>

In [18]:
list(kf.split(X))

[(array([2, 3]), array([0, 1])),
 (array([0, 1, 3]), array([2])),
 (array([0, 1, 2]), array([3]))]

In [19]:
for id_, (train_index, test_index) in enumerate(kf.split(X)):
    print(f'Experiment {id_}')
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("X")
    print(X_train, X_test, end='\n')
    print("y")
    print(y_train, y_test, end='\n')

Experiment 0
TRAIN: [2 3] TEST: [0 1]
X
[[102 204]
 [306 408]] [[101 202]
 [303 404]]
y
[3 4] [1 2]
Experiment 1
TRAIN: [0 1 3] TEST: [2]
X
[[101 202]
 [303 404]
 [306 408]] [[102 204]]
y
[1 2 4] [3]
Experiment 2
TRAIN: [0 1 2] TEST: [3]
X
[[101 202]
 [303 404]
 [102 204]] [[306 408]]
y
[1 2 3] [4]


## Leave-one-out техніка
Варіант кросвалідації, коли кількість розбиттів дорівнює кількості екземплярів в даних.

In [20]:
from sklearn.model_selection import LeaveOneOut

X = np.array([[101, 202], [303, 404], [102, 204], [306, 408], [404, 202]])
y = np.array([1, 2, 3, 4, 5])
loo = LeaveOneOut()

for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 2 3 4] TEST: [0]
TRAIN: [0 2 3 4] TEST: [1]
TRAIN: [0 1 3 4] TEST: [2]
TRAIN: [0 1 2 4] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]


## Стратифікована k-кратна (Stratified k-fold)

Ця техніка крос-валідації використовується для збереження пропорції класів у кожній складці (fold), що особливо важливо для незбалансованих наборів даних.

In [21]:
from sklearn.model_selection import StratifiedKFold

X = np.array([[101, 202], [102, 204], [303, 404], [306, 408], [101, 202], [505, 102]])
y = np.array([0, 0, 1, 1, 0, 1])
skf = StratifiedKFold(n_splits=3)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 3 4 5] TEST: [0 2]
TRAIN: [0 2 4 5] TEST: [1 3]
TRAIN: [0 1 2 3] TEST: [4 5]


# Пошук оптимальних гіперпараметрів

Завантажимо набір даних.

In [22]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split


def get_indices(X):
    train_full_id, test_id = train_test_split(X.index, test_size=test_size, shuffle=True, random_state=42)
    train_id, val_id = train_test_split(train_full_id, test_size=test_size, shuffle=True, random_state=42)
    return train_id, val_id, test_id

def get_metrics(y_true, y_predicted, n_digits=5):
    params = dict(y_true=y_true, y_pred=y_predicted)
    mse = mean_squared_error(**params)
    rmse = root_mean_squared_error(**params)
    return dict(mse=round(mse, n_digits), rmse=round(rmse, n_digits))

In [23]:
dataset = fetch_california_housing()
df = pd.DataFrame(dataset['data'], columns = dataset['feature_names'])
target_col_name = dataset['target_names'][0]
df[target_col_name] = dataset['target']
del dataset

In [24]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [25]:
X = df.drop([target_col_name],axis=1)
y = df[target_col_name]

In [26]:
test_size = 0.2

train_id, val_id, test_id = get_indices(X)

In [27]:
len(train_id), len(val_id), len(test_id)

(13209, 3303, 4128)

## Пошук по сітці (Grid search)

Пошук по сітці (Grid search) - це метод для налаштування гіперпараметрів моделі машинного навчання. Він передбачає перебір усіх можливих комбінацій заданих гіперпараметрів для визначення найкращої комбінації, яка забезпечує найвищу продуктивність моделі.

In [ ]:
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
degree = 2
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=degree)),
    ('model', Lasso())
])

In [ ]:
?GridSearchCV

In [ ]:
np.arange(0,10,0.1).shape

(100,)

In [ ]:
search = GridSearchCV(
    pipeline,
    {'model__alpha':np.arange(0,10,0.1)},
    cv = 5,
    scoring="neg_mean_squared_error",
    verbose=3
)

In [ ]:
search.fit(X.loc[train_id], y.loc[train_id])

In [ ]:
search.best_params_

{'model__alpha': 0.1}

In [ ]:
search.best_estimator_

array([2.01108779, 1.26290534, 2.07831121, ..., 1.90289166, 1.93371759,
       1.91129538])

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

Видимо, серед коефіцієнтів у нас вийшло багато нулів і фактично значущими залишилися лише декілька.

In [ ]:
coefficients.round(2)

array([ 0.  ,  0.71,  0.1 , -0.  , -0.  , -0.  , -0.  , -0.  , -0.  ,
       -0.  ,  0.  ,  0.  , -0.  , -0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  ,  0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  , -0.  , -0.  , -0.  ,  0.  ,  0.  , -0.  , -0.  , -0.  ,
        0.  ,  0.  , -0.  , -0.  , -0.  , -0.  , -0.02,  0.  ,  0.  ])

Давайте дізнаємось, які коефіцієнти є найбільш значущими:

In [ ]:
# dir(search.best_estimator_.named_steps['poly_features'])

In [ ]:
feature_names = search.best_estimator_.named_steps['poly_features'].get_feature_names_out()
coef_df = pd.DataFrame(zip(feature_names, coefficients.round(3)), columns=['feature_name', 'coef'])

In [ ]:
coef_df.sort_values('coef', ascending=False)[:10]

,feature_name,coef
1,x0,0.706
2,x1,0.105
33,x3 x6,-0.000
24,x2^2,0.000
25,x2 x3,-0.000
26,x2 x4,0.000
27,x2 x5,0.000
28,x2 x6,-0.000
29,x2 x7,-0.000
30,x3^2,-0.000


Словник з назвами ознак:

In [ ]:
X.columns.to_frame().reset_index(drop=True).to_dict()[0]

{0: 'MedInc',
 1: 'HouseAge',
 2: 'AveRooms',
 3: 'AveBedrms',
 4: 'Population',
 5: 'AveOccup',
 6: 'Latitude',
 7: 'Longitude'}

Видно, що все одно найважливіша ознака - MedInc без будь-яких ступенів.

In [ ]:
search.best_score_

-0.6640858768504454

In [ ]:
best_model = search.best_estimator_

y_train_pred = best_model.predict(X.loc[train_id])
y_val_pred = best_model.predict(X.loc[val_id])
train_metrics = get_metrics(y.loc[train_id], y_train_pred)
val_metrics = get_metrics(y.loc[val_id], y_val_pred)

In [ ]:
train_metrics, val_metrics

({'mse': 0.66241, 'rmse': 0.81389}, {'mse': 0.69114, 'rmse': 0.83135})

## Випадковий пошук (Random Search)

Випадковий пошук - це метод гіперпараметричної оптимізації, який випадковим чином вибирає комбінації гіперпараметрів з визначеного простору.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = dict()

# значения для alpha: 100 значений мужду e^-5 и e^5
params['alpha'] =  np.logspace(-5, 5, 100, endpoint=True)

# значения для l1_ratio: 100 значений между 0 и 1
params['l1_ratio'] = np.arange(0, 1, 0.01)

In [ ]:
len(params['alpha'])*len(params['l1_ratio'])

10000

In [ ]:
model = ElasticNet()

In [ ]:
?RandomizedSearchCV

Які стандартні методи оцінки якості моделі доступні в `RandomizedSearchCV` та `GridSearchCV`: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
search = RandomizedSearchCV(
    model,
    params,
    n_iter = 100,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=3,
    refit=True
)

In [ ]:
search.fit(X.loc[train_id], y.loc[train_id])

In [ ]:
display(search.best_estimator_, search.best_score_)

ElasticNet(alpha=0.00210490414451202, l1_ratio=0.18)

-0.5162179630417014

In [ ]:
ElasticNet()

ElasticNet()

In [ ]:
best_model = search.best_estimator_

y_train_pred = best_model.predict(X.loc[train_id])
y_val_pred = best_model.predict(X.loc[val_id])
train_metrics = get_metrics(y.loc[train_id], y_train_pred)
val_metrics = get_metrics(y.loc[val_id], y_val_pred)

In [ ]:
train_metrics, val_metrics

({'mse': 0.51302, 'rmse': 0.71626}, {'mse': 0.53882, 'rmse': 0.73405})

Нам вдалось якісно покращити модель. Ура!